In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import requests

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# If running in Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls

drive/  sample_data/


In [ ]:
cd drive/MyDrive/RecSys_KB/Aleksis

/content/drive/MyDrive/RecSys_KB/Aleksis


In [ ]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.3 MB/s eta 0:00:00


In [ ]:
def find_dbpedia_nodes(tag, confidence=0.5):
    """Find corresponding DBpedia node. """
    URL = "https://api.dbpedia-spotlight.org/en/annotate"
    PARAMS = {'text': tag, 'confidence' : confidence}
    HEADERS = {'Accept': 'application/json'}
    try:
        r = requests.get(url = URL, params = PARAMS, headers=HEADERS)
        data = r.json()
        nodes = []
        if 'Resources' in data.keys():
            for r in data['Resources']:
                nodes.append(r['@URI'])
        else:
            nodes = []

        return nodes
    except Exception as e:
        return r

In [ ]:
text = """Blade Runner"""


find_dbpedia_nodes(text)

['http://dbpedia.org/resource/Blade_Runner']

In [ ]:
from rdflib import Graph, URIRef, RDF
# from rdflib.namespace import dbo
uri = URIRef('http://dbpedia.org/resource/Blade_Runner')

g = Graph()
g.parse(uri)

query_en = """
PREFIX dbo:<http://dbpedia.org/ontology/>
SELECT ?abstract
WHERE {
    ?a dbo:abstract ?abstract .
    FILTER (lang(?abstract) = 'en')
}"""
query = """
PREFIX dbo:<http://dbpedia.org/ontology/>
SELECT ?abstract
WHERE {
    ?a dbo:abstract ?abstract .
}"""

qres = g.query(query_en)
for row in qres:
    print(f"{row.abstract}")

Blade Runner is a 1982 science fiction film directed by Ridley Scott, and written by Hampton Fancher and David Peoples. Starring Harrison Ford, Rutger Hauer, Sean Young, and Edward James Olmos, it is an adaptation of Philip K. Dick's 1968 novel Do Androids Dream of Electric Sheep? The film is set in a dystopian future Los Angeles of 2019, in which synthetic humans known as replicants are bio-engineered by the powerful Tyrell Corporation to work on space colonies. When a fugitive group of advanced replicants led by Roy Batty (Hauer) escapes back to Earth, burnt-out cop Rick Deckard (Ford) reluctantly agrees to hunt them down. Blade Runner initially underperformed in North American theaters and polarized critics; some praised its thematic complexity and visuals, while others critiqued its slow pacing and lack of action. It later became a cult film, and has since come to be regarded as one of the all-time best science fiction films. Hailed for its production design depicting a high-tech b

In [ ]:
try:
    uri = 5
    raise Exception("ext")
except Exception as err:
    print(err)
uri

ext


5

In [ ]:
bad_requests = []

In [ ]:
def get_abstracts(nodes):
    abstracts = []
    abstracts_count = 0

    for node in nodes:
        try:
            uri = URIRef(node)
            g = Graph()
            g.parse(uri)
            qres = g.query(query_en)
        except Exception as err:
            bad_requests.append(node)
            print(err)
            print(node)
            continue

        if len(qres) < 1:
            qres = g.query(query)
        for row in qres:
            abstracts.append(row.abstract)
            abstracts_count += 1

    return abstracts, abstracts_count


In [ ]:
meta_data = {
        'abstracts' : [],
        'abstracts_count' : [],
}

In [ ]:
# meta_data

In [ ]:
count = 0
count_more_than_one = 0
for i, movie in enumerate(movies['title']):
    nodes = find_dbpedia_nodes(movie)
    if len(nodes) < 1 and movie[-1] == ')':
        nodes = find_dbpedia_nodes(movie[:-1] + " film)")
    if len(nodes) > 0:
        # print(movie)
        count += 1
    if len(nodes) > 1:
        # print(movie)
        count_more_than_one += 1

    curr_abstracts, curr_abstracts_count = get_abstracts(nodes)
    meta_data['abstracts'].append('<sep>'.join(curr_abstracts))
    meta_data['abstracts_count'].append(curr_abstracts_count)

    if i % 100 == 0:
        print(i, count, count_more_than_one)

0 0 0
100 67 13
200 134 17
HTTP Error 400: Bad Request
http://dbpedia.org/resource/Tom_&_Viv
300 206 27
400 272 34
500 337 38
HTTP Error 400: Bad Request
http://dbpedia.org/resource/Cats_&_Dogs
HTTP Error 400: Bad Request
http://dbpedia.org/resource/Oliver_&_Company
600 417 55
700 481 60
HTTP Error 400: Bad Request
http://dbpedia.org/resource/Secrets_&_Lies_(film)
800 551 67


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/usr/local/lib/python3.10/dist-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-027-01-16'


900 622 79
1000 688 88
1100 766 107
HTTP Error 400: Bad Request
http://dbpedia.org/resource/Amos_&_Andrew
HTTP Error 400: Bad Request
http://dbpedia.org/resource/Batman_&_Robin_(film)
1200 835 112
HTTP Error 400: Bad Request
http://dbpedia.org/resource/In_&_Out_(film)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/usr/local/lib/python3.10/dist-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-480'
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/usr/local/lib/python3.10/dist-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-480'


1300 896 119
HTTP Error 400: Bad Request
http://dbpedia.org/resource/Kurt_&_Courtney
1400 962 125


...

8800 6197 886


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/usr/local/lib/python3.10/dist-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-004'


8900 6267 892
9000 6352 913
HTTP Error 400: Bad Request
http://dbpedia.org/resource/Men_&_Chicken


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/usr/local/lib/python3.10/dist-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-100-07-12'
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/usr/local/lib/python3.10/dist-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-100-07-12'
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/rdflib/term.py", line 2119, in _cas

9100 6428 922
9200 6513 938
HTTP Error 400: Bad Request
http://dbpedia.org/resource/Ratchet_&_Clank_(2016_video_game)
9300 6588 947
HTTP Error 400: Bad Request
http://dbpedia.org/resource/Never_Stop_(Echo_&_the_Bunnymen_song)
9400 6668 958
HTTP Error 400: Bad Request
http://dbpedia.org/resource/Ethel_&_Ernest_(film)
9500 6751 971
9600 6829 988
HTTP Error 400: Bad Request
http://dbpedia.org/resource/Elsa_&_Fred_(2014_film)
9700 6912 1001


In [ ]:
meta_data_df = pd.DataFrame.from_dict(meta_data)

In [ ]:
meta_data_df.to_csv('meta_data_df.csv', encoding='utf-8')

In [ ]:
meta_data_df

,abstracts,abstracts_count
0,,0
1,Jumanji is a 1995 American fantasy adventure f...,1
2,Grumpier Old Men is a 1995 American romantic c...,1
3,,0
4,,0
...,...,...
9737,The Butler Bulldogs men's basketball team repr...,2
9738,"No Game No Life (Japanese: ノーゲーム・ノーライフ, Hepbur...",1
9739,Flint is a sedimentary cryptocrystalline form ...,1
9740,"Bungo Stray Dogs (Japanese: 文豪ストレイドッグス, Hepbur...",2


In [ ]:
movies[["abstracts", "abstracts_count"]] = meta_data_df
movies

,movieId,title,genres,abstracts,abstracts_count
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji is a 1995 American fantasy adventure f...,1
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men is a 1995 American romantic c...,1
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,,0
4,5,Father of the Bride Part II (1995),Comedy,,0
...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,The Butler Bulldogs men's basketball team repr...,2
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,"No Game No Life (Japanese: ノーゲーム・ノーライフ, Hepbur...",1
9739,193585,Flint (2017),Drama,Flint is a sedimentary cryptocrystalline form ...,1
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,"Bungo Stray Dogs (Japanese: 文豪ストレイドッグス, Hepbur...",2


In [ ]:
movies.iloc[0]['abstracts']

''

In [ ]:
movies.to_csv('ml-latest-small/movies.csv', encoding='utf-8', index=False)

In [ ]:
movies.to_csv('movies_with_abstracts.csv', encoding='utf-8', index=False)

In [ ]:
pd.read_csv('ml-latest-small/movies.csv', keep_default_na=False, encoding='utf-8')

,movieId,title,genres,abstracts,abstracts_count
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji is a 1995 American fantasy adventure f...,1
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men is a 1995 American romantic c...,1
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,,0
4,5,Father of the Bride Part II (1995),Comedy,,0
...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,The Butler Bulldogs men's basketball team repr...,2
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,"No Game No Life (Japanese: ノーゲーム・ノーライフ, Hepbur...",1
9739,193585,Flint (2017),Drama,Flint is a sedimentary cryptocrystalline form ...,1
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,"Bungo Stray Dogs (Japanese: 文豪ストレイドッグス, Hepbur...",2
